In [23]:
pip install google_patent_scraper

Note: you may need to restart the kernel to use updated packages.


In [24]:
# load csv to pandas
import pandas as pd

df = pd.read_csv (r'patentdatacleaned.csv')
df

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,CN-111251819-A,Vehicle-mounted air conditioner intelligent ad...,武汉理工大学,"胡杰, 宋洪干",03/01/2020,03/01/2020,09/06/2020,NaN,https://patents.google.com/patent/CN111251819A/en,NaN
1,CN-110809211-B,Method for actively reducing noise of earphone...,恒玄科技(北京)有限公司,"童伟峰, 张亮, 李倩, 徐明亮",08/01/2020,08/01/2020,19/05/2020,19/05/2020,https://patents.google.com/patent/CN110809211B/en,NaN
2,CN-111211872-A,PDCCH blind detection method for SNR mean valu...,重庆邮电大学,"张治中, 王玲, 杨钦雲, 胡玲玲",09/01/2020,09/01/2020,29/05/2020,NaN,https://patents.google.com/patent/CN111211872A/en,NaN
3,CN-110854460-B,"Battery cell discharging method, and battery f...",长兴太湖能谷科技有限公司,"吴建斌, 陈驰",09/01/2020,16/01/2020,14/04/2020,14/04/2020,https://patents.google.com/patent/CN110854460B/en,NaN
4,CN-111210144-A,Power generation risk management method and sy...,"南方电网科学研究院有限责任公司, 中国南方电网有限责任公司","周保荣, 黎小林, 赵文猛, 洪潮",02/01/2020,02/01/2020,29/05/2020,NaN,https://patents.google.com/patent/CN111210144A/en,NaN
...,...,...,...,...,...,...,...,...,...,...
11642,RU-199848-U1,DEVICE FOR COLLECTING AND TRANSMISSION OF SPAT...,Федеральное государственное бюджетное образова...,"Илья Николаевич Бондарев, Олег Геннадьевич Гво...",01/01/2020,01/01/2020,22/09/2020,22/09/2020,https://patents.google.com/patent/RU199848U1/ru,https://patentimages.storage.googleapis.com/d9...
11643,RU-2730451-C1,Средство для пилинга при гиперкератозе стоп,Федеральное государственное бюджетное учрежден...,"Александр Ливиевич Ураков, Михаил Юрьевич Алье...",10/01/2020,10/01/2020,24/08/2020,24/08/2020,https://patents.google.com/patent/RU2730451C1/ru,NaN
11644,RU-2736956-C1,Способ аэроэлектроразведки с применением легко...,"Общество с ограниченной ответственностью ""Геои...",Александр Вадимович Паршин,09/01/2020,09/01/2020,23/11/2020,23/11/2020,https://patents.google.com/patent/RU2736956C1/ru,https://patentimages.storage.googleapis.com/a8...
11645,RU-200098-U1,STEAM EMISSION SILENCER,Федеральное государственное бюджетное образова...,"Роман Евгеньевич Липантьев, Эльмира Нурулловна...",10/01/2020,10/01/2020,06/10/2020,06/10/2020,https://patents.google.com/patent/RU200098U1/ru,https://patentimages.storage.googleapis.com/68...


In [25]:
# Import packages 
import google_patent_scraper
from google_patent_scraper import scraper_class
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
database = client["meg_de"]
collection = database.patents_data


# Initialize scraper class 
scraper=scraper_class(return_abstract=True) 

result_list = (list(df["result link"]))

abstract_list = list()

for index,row in df.iterrows():
    patent = row['id'].replace("-","")
    url = row["result link"]
    soup = BeautifulSoup(requests.get(url).content,'html.parser')
    patent_1_parsed = scraper.get_scraped_data(soup,patent,url)
    abstract_list.append(patent_1_parsed['abstract_text'].strip())
    #print(patent_1_parsed['abstract_text'])


In [18]:
df2 = df.assign(abstract = abstract_list) 
df2.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link,abstract
0,CN-111251819-A,Vehicle-mounted air conditioner intelligent ad...,武汉理工大学,"胡杰, 宋洪干",03/01/2020,03/01/2020,09/06/2020,NaN,https://patents.google.com/patent/CN111251819A/en,NaN,The invention discloses a vehicle-mounted air ...
1,CN-110809211-B,Method for actively reducing noise of earphone...,恒玄科技(北京)有限公司,"童伟峰, 张亮, 李倩, 徐明亮",08/01/2020,08/01/2020,19/05/2020,19/05/2020,https://patents.google.com/patent/CN110809211B/en,NaN,The disclosure relates to a method for activel...
2,CN-111211872-A,PDCCH blind detection method for SNR mean valu...,重庆邮电大学,"张治中, 王玲, 杨钦雲, 胡玲玲",09/01/2020,09/01/2020,29/05/2020,NaN,https://patents.google.com/patent/CN111211872A/en,NaN,The invention relates to a PDCCH blind detecti...
3,CN-110854460-B,"Battery cell discharging method, and battery f...",长兴太湖能谷科技有限公司,"吴建斌, 陈驰",09/01/2020,16/01/2020,14/04/2020,14/04/2020,https://patents.google.com/patent/CN110854460B/en,NaN,"A battery cell discharging method, a battery f..."
4,CN-111210144-A,Power generation risk management method and sy...,"南方电网科学研究院有限责任公司, 中国南方电网有限责任公司","周保荣, 黎小林, 赵文猛, 洪潮",02/01/2020,02/01/2020,29/05/2020,NaN,https://patents.google.com/patent/CN111210144A/en,NaN,The invention discloses a power generation ris...


In [19]:
patents_list = df2.to_dict('records')

In [20]:
collection.insert_many(patents_list)